## Fourier Series

In Fourier analysis, we are seeking to express a periodic function $g(t)=g(t+T)$ in terms of the sum of a sine and cosine functions, i.e.,

$g(t) = \frac{A_0}{2} + A_1 \cos\left( \frac{2 \pi t}{T} \right) + A_2 \cos\left( \frac{4 \pi t}{T} \right) + A_3 \cos\left( \frac{6 \pi t}{T} \right) + \cdots + B_1 \sin\left( \frac{2 \pi t}{T} \right) + B_2 \sin\left( \frac{4 \pi t}{T} \right) + B_3 \sin\left( \frac{6 \pi t}{T} \right) + \cdots$

Or more compactly,

$g(t) = \frac{A_0}{2} + \sum_{n=1}^\infty \left[A_n \cos\left( \frac{2 \pi n t}{T} \right)+ B_n \sin\left( \frac{2 \pi n t}{T} \right) \right]$

The following activity will help build intuition for finding the right coefficients for a Fourier Series: https://phet.colorado.edu/sims/html/fourier-making-waves/latest/fourier-making-waves_en.html

Click on the "wave game" tab. Your goal is to find the relative strengths of various sine curves that need to be added together to reconstruct this curve. Start with level 1 and go through all the levels.

Is there a way to find the coefficients mathematically? The mission is to find out what the coefficients $A_n$ and $B_n$ are. We can write

$ A_n = \frac{2}{T} \int_0^T g(t) \cos \left( \frac{2 \pi n t}{T} \right) dt$

and

$ B_n = \frac{2}{T} \int_0^T g(t) \sin \left( \frac{2 \pi n t}{T} \right) dt$.

## Fourier Transform

The Fourier series requires _g(t)_ to be a periodic function. The concept of a _Fourier Transform_ removes this restriction by thinking of a non-periodic function as a periodic function with $T \rightarrow \infty$. While the reconstruction of a periodic function can be accomplished by summing together sines and cosines at a discrete set of frequencies, the reconstruction of a non-periodic function requires the summation over a continuous set of all possible frequencies. This means that we can work out an entire function of "coefficients":

$\tilde{g}(f) = \int_{-\infty}^\infty e^{-i2 \pi ft} g(t) dt$

The function $\tilde{g}(f)$ is known as the Fourier Transform of $g(t)$.

## Numerical Fourier Transforms

In this exercise we'll explore how one can take Fourier transforms numerically using Python. The Discrete Fourier Transform (DFT) of a discrete set of points $g_1 = g(t_1)$, $g_2 = g(t_2) $, ..., $g_N = g(t_N = T)$ is defined to be

$\tilde{g}_n \equiv \sum_{m=1}^N g_m \exp \left( \frac{-i 2 \pi m n }{N} \right)$

which tells us how much of each complex exponential is needed to reconstruct the series of $g_m$ values. This reconstruction is accomplished by the inverse Discrete Fourier transform, given by

$g_m \equiv \frac{1}{N} \sum_{n=1}^N \tilde{g}_n \exp \left( \frac{+i 2 \pi m n }{N} \right)$.

Python has built-in functions to compute Fourier transforms. They can be found under `np.fft`, where "FFT" stands for "Fast Fourier Transform". The FFT is mathematically the same as the DFT, but uses various internal computing tricks to make things "fast". Let's try it out!

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
SAMPLE_RATE = 44100  # Hertz
DURATION = 5  # Seconds
AMPLIFICATION = 1.0
DC_OFFSET = 0.0

def generate_sine_wave(freq, sample_rate, duration, amplification, dc_offset):
    x = np.linspace(0, duration, sample_rate * duration, endpoint=False)
    frequencies = x * freq
    # 2pi because np.sin takes radians
    y = amplification*np.sin((2 * np.pi) * frequencies) + dc_offset
    return x, y

# Generate a 2 hertz sine wave that lasts for 5 seconds
x, y = generate_sine_wave(2, SAMPLE_RATE, DURATION, AMPLIFICATION, DC_OFFSET)
plt.plot(x, y)
plt.xlabel('Time (s)')

In [ ]:
N = SAMPLE_RATE * DURATION

yf = np.fft.fftshift(np.fft.fft(y))
xf = np.fft.fftshift(np.fft.fftfreq(N, 1 / SAMPLE_RATE))

plt.plot(xf, np.abs(yf))
plt.xlabel('Frequency (Hz)')
plt.xlim([-5,5])

### Top Hat Example

The top hat function is a function is a function that is 1 close to the origin but zero everywhere else.

In [ ]:
def tophat(t,a):
    out = t.copy()
    for i,elem in enumerate(t):
        if abs(elem) > a:
            out[i] = 0.
        else:
            out[i] = 1.
    return out

In [ ]:
m_coords = np.arange(-1024, 1024)
gt = tophat(m_coords,150)

f, ax = plt.subplots(figsize=(9,6))
ax.plot(m_coords,gt,lw=2,c='red')
ax.set_xlabel(r"$m$")
ax.set_ylabel(r"$g_m$")
ax.set_ylim(-0.2, 1.2)
plt.show()

Here, you'll see that rather than having $m$ start at zero, I decided to have it be centered around zero, so that the $g_m$'s start from $m= -N/2$ rather than $m=0$. This is typical of a lot of our data. However, the slightly irritating thing about most numerical FFT routines is that they assume we're starting at $m=0$ Luckily, `numpy` provides a handy routine called "fftshift" that can be used to put our data into the right format.

In [ ]:
f, ax = plt.subplots(figsize=(9,6))
ax.plot(np.fft.fftshift(gt),lw=2,c='red')
ax.set_xlim(0,2048)
ax.set_ylim(-0.2,1.2)
ax.set_xlabel("Array index")
plt.show()

Indeed, the first element of the array now corresponds to $g_0$. But notice how the part of the function immediately to the left of $m=0$ is now on the far right! It turns out that FFTs assume that axes "wrap around". So in our new ordering scheme, the data goes from $m=0$ to $m=-N/2$, and then goes from $t=-N/2$ back up to $t=0$. Thus, the left half of the bump at appears on the far right. Said differently, FFTs assume that our function is periodic. In this way, FFTs are actually quite closely related to Fourier *Series* (which, remember, are for like Fourier transforms for periodic functions).

The FFT algorithm also orders the *output* of the FFT with $\tilde{g}_0$ as the first element. Thus, the output needs to be adjusted too, and the whole process is given by an initial np.fft.fftshift, the actual FFT using np.fft.fft, and the reordering of the final result using another np.fft.fftshift:

In [ ]:
gtilde = np.fft.fftshift(np.fft.fft(np.fft.fftshift(gt)))

f, ax = plt.subplots(figsize=(9,6))
ax.plot(gtilde.real,lw=2,c='red')
ax.plot(gtilde.imag,lw=2,c='blue')
ax.set_xlim(512,1536)
ax.set_xlabel("n")
ax.set_ylabel(r"$\widetilde{g}_n$")
plt.show()

This is known as a <b> sinc </b> function. This looks like the function $\tilde{g} (f) = \sin (2 \pi f) / \pi f$ that we would've predicted by doing the analytic version of the Fourier transform. But whereas the pencil-and-paper version gave us a continuous function of $f$, here we have a bunch of values in an array, and we need to know which value of $f$ each element of the output array corresonds to. It turns out that numpy has a function called np.fft.fftfreq for this. As input, it needs to know the length of the original input array to the FFT, as well as the spacing in $f$ that is assumed there.

### The Effect of Width

Let's look at what happens when we change the _width_ of the tophat function.  Here we're going to have to take seriously how to get the x-axis of the Fourier transform labeled properly so we can interpret it.

In [ ]:
def gt(t,a):
    out = t.copy()
    for i,elem in enumerate(t):
        if abs(elem) > a:
            out[i] = 0.
        else:
            out[i] = 1.
    return out

def gtilde(f,a):
    return a* np.sinc(f * a)

In [ ]:
t, dt = np.linspace(-10.,10.,1000,retstep=True,endpoint=False)

f, ax = plt.subplots(figsize=(9,6))
ax.plot(t,gt(t,1.),lw=2,c='blue',label='Narrow')
ax.plot(t,gt(t,2.5),lw=2,c='red',label='Medium')
ax.plot(t,gt(t,5.),lw=2,c='black',label='Wide')
ax.axhline(0, color='black',ls='--')
ax.axvline(0, color='black',ls='--')
ax.set_ylim(-0.2, 1.2)
plt.legend()

In [ ]:
# Make the frequencies corresponding to the time sampling
freq = np.fft.fftfreq(n=len(t),d=dt)

f, ax = plt.subplots(figsize=(9,6))
ax.plot(freq,gtilde(freq,1.),lw=2,c='blue',label='Narrow')
ax.plot(freq,gtilde(freq,2.5),lw=2,c='red',label='Medium')
ax.plot(freq,gtilde(freq,5.),lw=2,c='black',label='Wide')
ax.axhline(0, color='black',ls='--')
ax.axvline(0, color='black',ls='--')
ax.set_xlim(-5,5)
plt.legend()
plt.show()

## Discrete Sampling

When we talk about radio data, our ADC will give us discrete points in a timestream. Generally, to make the functions we're working with look continuous, we typically pick a fine grid of $t$ values to plot. Let's go from $t=0$ to $t=1$.

Note that $T=0.1$ corresponds to a frequency of $f$ = 10 Hz.

In [ ]:
t = np.linspace(0., 1.0, 400)
g_fct = 5. * np.sin(2. * np.pi * t / 0.1)

f, ax = plt.subplots(figsize=(9,6))
ax.plot(t,g_fct,lw=2,c='black')
ax.plot(t,g_fct,'o',c='red')
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$g(t)$")

What do you expect the fft to look like? What happens if we sample at a lower rate? (Try this yourself - change the last integer in the t_low variable that controls the number of samples.)

In [ ]:
t_low = np.linspace(0., 1.0, 80)
g_fct_low = 5. * np.sin(2. * np.pi * t_low / 0.1)

f, ax = plt.subplots(figsize=(9,6))
ax.plot(t,g_fct,lw=2,c='black')
ax.plot(t_low,g_fct_low,'o',c='red')
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$g(t)$")

What about an even lower rate?

In [ ]:
t_lower = np.linspace(0., 1.0, 9)
g_fct_lower = 5. * np.sin(2. * np.pi * t_lower / 0.1)
g_fct_alias = 5. * np.sin(2. * np.pi * t / 0.5)

f, ax = plt.subplots(figsize=(9,6))
ax.plot(t,g_fct,lw=2,c='black')
ax.plot(t,g_fct_alias,'--',lw=2,c='blue')
ax.plot(t_lower,g_fct_lower,'o',c='red')
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$g(t)$")

Eventually, we will sample low enough that the data (the red dots) does not give us enough information about the signal for us to accurately recreate it. The above example shows the same 10 Hz signal, sampled at a low enough sampling rate that the data could be confused for a 2 Hz signal instead. This is known as _aliasing_.

Generally, a maximum of 0.5 cycles / sample will give us the samples we need to recreate the signal. This is known as the _Nyquist frequency_.

In [ ]:
t_nyq = np.linspace(0.025, 0.975, 20)
g_fct_nyq = 5. * np.sin(2. * np.pi * t_nyq / 0.1)

f, ax = plt.subplots(figsize=(9,6))
ax.plot(t,g_fct,lw=2,c='black')
ax.plot(t_nyq,g_fct_nyq,'o',c='red')
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"$g(t)$")

## In-band and Out-of-band Saturation

If we get signals that are high enough in amplitude, there is a chance that we will saturate our amplifier or our ADC. What does this look like?

In [ ]:
def clipped(y):
    T = 0.1
    #y = a*np.sin((2 * np.pi) * x / T)
    for i in range(len(y)):
        if y[i] > 1.0:
            y[i]= 1.0
        if y[i] < -1.0:
            y[i] = -1.0
    return y

In [ ]:
SAMPLE_RATE = 44100  # Hertz
DURATION = 5  # Seconds
AMPLIFICATION = 1.3
DC_OFFSET = 0.0

# Generate a 2 hertz sine wave that lasts for 5 seconds
x, y = generate_sine_wave(2, SAMPLE_RATE, DURATION, AMPLIFICATION, DC_OFFSET)

# Clip anything over 1.0 amplitude
y = clipped(y)

plt.plot(x, y)
plt.xlabel('Time (s)')

Try changing the AMPLIFICATION value and notice how over 1.0, our sine wave starts looking like a square wave. Why is this bad?

Even if our signal is only saturated at a specific frequency, this will affect our signal as a whole. For example:

In [ ]:
AMPLIFICATION_1 = 1.1
AMPLIFICATION_2 = 0.3

_, tone_1 = generate_sine_wave(2, SAMPLE_RATE, DURATION, AMPLIFICATION_1, DC_OFFSET)
_, tone_2 = generate_sine_wave(10, SAMPLE_RATE, DURATION, AMPLIFICATION_2, DC_OFFSET)

mixed_tone = tone_1 + tone_2
mixed_tone = clipped(mixed_tone)

plt.plot(x, mixed_tone)
plt.xlabel('Time (s)')

## DC Offset

We've been working with perfect sine waves centered around 0. However, most radio data have a "DC offset" value, typically due to a noise floor.

Let's model what a DC offset what would like in Fourier space.

In [ ]:
SAMPLE_RATE = 44100  # Hertz
DURATION = 5  # Seconds
AMPLIFICATION = 1.0
DC_OFFSET = 0.5

# Generate a 2 hertz sine wave that lasts for 5 seconds
x, y = generate_sine_wave(2, SAMPLE_RATE, DURATION, AMPLIFICATION, DC_OFFSET)
plt.plot(x, y)
plt.xlabel('Time (s)')

In [ ]:
N = SAMPLE_RATE * DURATION

yf = np.fft.fftshift(np.fft.fft(y))
xf = np.fft.fftshift(np.fft.fftfreq(N, 1 / SAMPLE_RATE))

plt.plot(xf, np.abs(yf))
plt.xlabel('Frequency (Hz)')
plt.xlim([-5,5])

Why do you see the spike at 0?